# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,16.54,79,58,1.79,PT,1701922761
1,1,albany,42.6001,-73.9662,-2.46,82,39,0.89,US,1701922689
2,2,stanley,54.8680,-1.6985,3.20,86,75,4.12,GB,1701922806
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.03,79,14,6.51,SH,1701922722
4,4,adamstown,-25.0660,-130.1015,22.70,83,66,5.96,PN,1701922708


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.01,
    color = "City"
)

# Display the map
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [43]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_df = city_data_df.dropna(how = 'all')

# Drop any rows with null values

# Display sample data
new_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,16.54,79,58,1.79,PT,1701922761
1,1,albany,42.6001,-73.9662,-2.46,82,39,0.89,US,1701922689
2,2,stanley,54.8680,-1.6985,3.20,86,75,4.12,GB,1701922806
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.03,79,14,6.51,SH,1701922722
4,4,adamstown,-25.0660,-130.1015,22.70,83,66,5.96,PN,1701922708


### Step 3: Create a new DataFrame called `hotel_df`.

In [56]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_data = (new_city_df)
hotel_df = pd.DataFrame(hotel_data)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,ribeira grande,38.5167,-28.7000,16.54,79,58,1.79,PT,1701922761,
1,1,albany,42.6001,-73.9662,-2.46,82,39,0.89,US,1701922689,
2,2,stanley,54.8680,-1.6985,3.20,86,75,4.12,GB,1701922806,
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.03,79,14,6.51,SH,1701922722,
4,4,adamstown,-25.0660,-130.1015,22.70,83,66,5.96,PN,1701922708,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 50000
params = {
 "categories":"hotels",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city = row["City"]
    latitude = row["Lat"]
    longitude = row["Lon"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


NameError: name 'hotel_df' is not defined

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)